# Tree Methods Consulting Project

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree_cons').getOrCreate()

In [0]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import year, month, dayofmonth
#rfcModel.featureImportances

In [0]:
df = sqlContext.sql("SELECT * FROM dog_food_csv")
df.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
summary| A| B| C| D| Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
 count| 490| 490| 490| 490| 490|
 mean| 5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
 stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
 min| 1| 1| 5.0| 1| 0.0|
 max| 10| 10| 14.0| 10| 1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+

In [0]:
df.columns

Out[6]: ['A', 'B', 'C', 'D', 'Spoiled']

In [0]:
ass = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [0]:
op = ass.transform(df)

In [0]:
fdf = op.select('features','Spoiled')
fdf.show()

+-------------------+-------+
 features|Spoiled|
+-------------------+-------+
 [4.0,2.0,12.0,3.0]| 1.0|
 [5.0,6.0,12.0,7.0]| 1.0|
 [6.0,2.0,13.0,6.0]| 1.0|
 [4.0,2.0,12.0,1.0]| 1.0|
 [4.0,2.0,12.0,3.0]| 1.0|
[10.0,3.0,13.0,9.0]| 1.0|
 [8.0,5.0,14.0,5.0]| 1.0|
 [5.0,8.0,12.0,8.0]| 1.0|
 [6.0,5.0,12.0,9.0]| 1.0|
 [3.0,3.0,12.0,1.0]| 1.0|
 [9.0,8.0,11.0,3.0]| 1.0|
[1.0,10.0,12.0,3.0]| 1.0|
[1.0,5.0,13.0,10.0]| 1.0|
[2.0,10.0,12.0,6.0]| 1.0|
[1.0,10.0,11.0,4.0]| 1.0|
 [5.0,3.0,12.0,2.0]| 1.0|
 [4.0,9.0,11.0,8.0]| 1.0|
 [5.0,1.0,11.0,1.0]| 1.0|
[4.0,9.0,12.0,10.0]| 1.0|
 [5.0,8.0,10.0,9.0]| 1.0|
+-------------------+-------+
only showing top 20 rows

In [0]:
train, test = fdf.randomSplit([.7,.3])

In [0]:
dtc = DecisionTreeClassifier(labelCol='Spoiled')
rfc = RandomForestClassifier(labelCol='Spoiled', numTrees=100)
gbt = GBTClassifier(labelCol='Spoiled')

In [0]:
dtcModel = dtc.fit(train)
rfcModel = rfc.fit(train)
gbtModel = gbt.fit(train)

In [0]:
dtcPreds = dtcModel.transform(test)
rfcPreds = rfcModel.transform(test)
gbtPreds = gbtModel.transform(test)

In [0]:
binEval=BinaryClassificationEvaluator(labelCol='Spoiled')
binEval2=BinaryClassificationEvaluator(labelCol='Spoiled', rawPredictionCol='prediction')

In [0]:
print('DTC')
print(binEval.evaluate(dtcPreds))
print('RFC')
print(binEval.evaluate(rfcPreds))
print('GBT')
print(binEval2.evaluate(gbtPreds))

DTC
1.0
RFC
1.0
GBT
1.0

In [0]:
acc = MulticlassClassificationEvaluator(labelCol='Spoiled', metricName='accuracy')

In [0]:
dtcAcc = acc.evaluate(dtcPreds)
rfcAcc = acc.evaluate(rfcPreds)
gbtAcc = acc.evaluate(gbtPreds)
print('DTC')
print(dtcAcc)
print('RFC')
print(rfcAcc)
print('GBT')
print(gbtAcc)

DTC
1.0
RFC
1.0
GBT
1.0

In [0]:
fdtcModel = dtc.fit(fdf)
frfcModel = rfc.fit(fdf)
fgbtModel = gbt.fit(fdf)

In [0]:
dtcModel.featureImportances

Out[34]: SparseVector(4, {1: 0.0022, 2: 0.9601, 3: 0.0377})

In [0]:
rfcModel.featureImportances

Out[35]: SparseVector(4, {0: 0.0313, 1: 0.0306, 2: 0.9081, 3: 0.0299})

In [0]:
gbtModel.featureImportances

Out[36]: SparseVector(4, {0: 0.0568, 1: 0.0503, 2: 0.8321, 3: 0.0608})

In [0]:
fdtcModel.featureImportances

Out[37]: SparseVector(4, {1: 0.0019, 2: 0.9832, 3: 0.0149})

In [0]:
frfcModel.featureImportances

Out[38]: SparseVector(4, {0: 0.0213, 1: 0.0181, 2: 0.9388, 3: 0.0218})

In [0]:
fgbtModel.featureImportances

Out[39]: SparseVector(4, {0: 0.0296, 1: 0.0383, 2: 0.8286, 3: 0.1034})